In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
X_test_full = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
# Remove rows with missing target, separate target from predictors


X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [85]:
input_shape = [X_train.shape[1]]
print(input_shape)

[227]


In [86]:
from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE: define the model given in the diagram
model = keras.Sequential([layers.BatchNormalization(input_shape = input_shape),
                          layers.Dense(256, activation = 'relu'),
                          layers.BatchNormalization(),
                          layers.Dropout(0.3),
                          layers.Dense(256, activation = 'relu'),
                          layers.BatchNormalization(),
                          layers.Dropout(0.3),
                          layers.Dense(1, activation = 'sigmoid')])


In [87]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],)

In [88]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],
)
prediction = model.predict(X_test)
print(prediction)

Epoch 1/200
3/3 [==============================] - 2s 176ms/step - loss: -2677.0171 - binary_accuracy: 0.0000e+00 - val_loss: -12092.5498 - val_binary_accuracy: 0.0000e+00
Epoch 2/200
3/3 [==============================] - 0s 33ms/step - loss: -14903.7842 - binary_accuracy: 0.0000e+00 - val_loss: -24944.5781 - val_binary_accuracy: 0.0000e+00
Epoch 3/200
3/3 [==============================] - 0s 36ms/step - loss: -27916.7539 - binary_accuracy: 0.0000e+00 - val_loss: -38674.5156 - val_binary_accuracy: 0.0000e+00
Epoch 4/200
3/3 [==============================] - 0s 34ms/step - loss: -41838.8281 - binary_accuracy: 0.0000e+00 - val_loss: -53369.7031 - val_binary_accuracy: 0.0000e+00
Epoch 5/200
3/3 [==============================] - 0s 38ms/step - loss: -56663.4648 - binary_accuracy: 0.0000e+00 - val_loss: -69118.9922 - val_binary_accuracy: 0.0000e+00
Epoch 6/200
3/3 [==============================] - 0s 34ms/step - loss: -72697.6875 - binary_accuracy: 0.0000e+00 - val_loss: -85992.5234 - 